In [63]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx
import random
from icecream import ic
from tqdm.auto import tqdm
from matplotlib import pyplot as plt
from itertools import accumulate
import functools


In [64]:
logging.basicConfig(level=logging.DEBUG)

## Nearest Neighbor (Greedy)

In [65]:
def nearest_neighbor_tsp(start_city_index=0):
    """
    Solves the TSP using the Nearest Neighbour Greedy algorithm.
    :param start_city_index: Index of the starting city
    :return: List of the order of the visited cities and total length of the route
    """
    dist_matrix=DIST_MATRIX.copy()
    visited = np.full(len(CITIES), False)  
    city = start_city_index  
    visited[city] = True

    tsp = [city]  

    while not np.all(visited):
        dist_matrix[:, city] = np.inf
        closest_city = np.argmin(dist_matrix[city])
        visited[closest_city] = True
        tsp.append(int(closest_city))
        city = closest_city

    tsp.append(start_city_index)
    
    total_distance = tsp_cost(tsp,DIST_MATRIX)
    
    return tsp, total_distance

In [66]:
def counter(fn):
    """Simple decorator for counting number of calls"""

    @functools.wraps(fn)
    def helper(*args, **kargs):
        helper.calls += 1
        return fn(*args, **kargs)

    helper.calls = 0
    return helper


In [ ]:
def valid(tsp):
    """
    It verifies the validity of a TSP route.
    """
    tsp = np.array(tsp)
    
    if tsp[0] != tsp[-1]:
        return False
    
    # Verify that all cities are visited exactly once.
    cities_visited = tsp[:-1]
    expected_cities = np.arange(len(CITIES))
    
    return (len(np.unique(cities_visited)) == len(CITIES) and 
            np.all(np.isin(expected_cities, cities_visited)))
@counter
def tsp_cost(route, dist_matrix):
    """It calculates the cost of the route and its validity."""
    route = np.array(route)
    # Calculates distances between consecutive cities.
    total_distance = np.sum([dist_matrix[route[i], route[i+1]] 
                           for i in range(len(route)-1)])
    return (total_distance, valid(route))

## Inversion mutation

In [68]:
def tweak(tsp):
    """Performs an inversion mutation on a TSP solution"""
    tsp = np.array(tsp)
    # Select two random points in the path to perform the inversion.
    a, b = sorted(np.random.choice(range(1, len(tsp) - 1), 2, replace=False))
    # Reverses the sub-path between the two selected points.
    tsp[a:b+1] = np.flip(tsp[a:b+1])
    return tsp.tolist()


##  Simulated Annealing (SA) by Initial Population.

In [69]:
def simulated_annealing_population(dist_matrix, initial_temp, cooling_rate, stop_temp, population_size=100):
    """Simulated Annealing algorithm to generate an initial population"""
    dist_matrix = np.array(dist_matrix)
     # Find an initial solution with the Greedy method.
    current_solution, current_cost = nearest_neighbor_tsp(start_city_index=0)
    best_solution = current_solution[:]
    best_cost = current_cost

    population = [(best_solution, best_cost)]
    temperature = initial_temp

    while temperature > stop_temp:
        new_solution = tweak(current_solution[:])
        new_cost = tsp_cost(new_solution, dist_matrix)

        if (new_cost[0] < current_cost[0] or 
            np.random.random() < np.exp((current_cost[0] - new_cost[0]) / temperature)):
            current_solution = new_solution
            current_cost = new_cost

            if new_cost[0] < best_cost[0]:
                best_solution = new_solution
                best_cost = new_cost
    
            population.append((current_solution, current_cost))
            
        temperature *= cooling_rate
    # Sort the population by cost and return the best 'population_size' individuals    
    population = np.array(population, dtype=object)
    sorted_indices = np.argsort([cost[0] for _, cost in population])
    population = population[sorted_indices]
    
    return population[:population_size].tolist()

In [70]:
def initial_population(pop_size):
    """Initialises the population using predefined parameters."""
    initial_temp = 1000
    cooling_rate = 0.992
    stop_temp = 1e-200
    # Generate initial population with Simulated Annealing.
    return simulated_annealing_population(DIST_MATRIX, initial_temp,
                                        cooling_rate, stop_temp, pop_size)


##  Crossover Inver-Over

In [71]:
def inver_over_crossover(parent1, parent2, num_iterations=1):
    """Apply inver-over crossover """
    parent1, parent2 = np.array(parent1), np.array(parent2)
    child = parent1.copy()
 # Perform a series of crossover iterations to create the child
    for _ in range(num_iterations):
        start_index = np.random.randint(1, len(child) - 2)
        start_city = child[start_index]
        # Randomly select a destination city from one of the parents.
        if np.random.random() < 0.5:
            valid_cities = child[1:-1][child[1:-1] != start_city]
            end_city = np.random.choice(valid_cities)
        else:
            valid_cities = parent2[1:-1][parent2[1:-1] != start_city]
            end_city = np.random.choice(valid_cities)
        # Get start and destination city locations in the son.
        start_pos = np.where(child == start_city)[0][0]
        end_pos = np.where(child == end_city)[0][0]
        
        if start_pos > end_pos:
            start_pos, end_pos = end_pos, start_pos
          # Reverse the sub-path between the two positions.
        child[start_pos:end_pos + 1] = np.flip(child[start_pos:end_pos + 1])
    # The child ends up in the same city as the starting city.
    if child[0] != child[-1]:
        child = np.append(child, child[0])
    
    return child.tolist()

In [72]:
def tournament_selection(population, k=5):
    """Tournament selection using NumPy."""
    # Randomly selects 'k' individuals from the population.
    selected_indices = np.random.choice(len(population), k, replace=False)
    selected = np.array(population, dtype=object)[selected_indices]
    # Returns the individual with the lowest cost among those selected.
    return min(selected, key=lambda x: x[1][0])  # x[1][0] accede al costo


## Inversion mutation

In [ ]:
def inversion_mutation(tsp):
    """Performs an inversion mutation """
    tsp = np.array(tsp)
    a, b = sorted(np.random.choice(range(1, len(tsp) - 1), 2, replace=False))
    tsp[a:b+1] = np.flip(tsp[a:b+1])
    return tsp.tolist()


## EA with adaptive mutation and multiple restart with simulated anneling.

In [ ]:

def evolutionary_algorithm_tsp_adaptive_mutation(
    
    dist_matrix,
    population_size=100, 
    tournament_size=10, 
    num_generations=10000, 
    initial_mutation_rate=0.1, 
    crossover_iterations=1,
    max_generations_without_improvement=500,
    mutation_increase_factor=1.5,
    mutation_decrease_factor=0.7,
    mutation_threshold=1.3, # Threshold for restarting
    initial_temp=1000,      
    cooling_rate=0.992,
    stop_temp=1e-200,

):
    """It solves the TSP using a Evolutionary Algorithm with adaptive mutation."""
    dist_matrix = np.array(dist_matrix)
    population = initial_population(population_size)
    current_mutation_rate = initial_mutation_rate

    best_solution, best_cost = min(population, key=lambda x: x[1][0])
    all_time_best_solution = best_solution
    all_time_best_cost = best_cost
    generations_without_improvement = 0
    

    for generation in tqdm(range(num_generations)):
        # Check if it is necessary to reboot with SA.
        if current_mutation_rate > mutation_threshold:
            print(f"\nMutation rate ({current_mutation_rate:.3f}) exceeded threshold ({mutation_threshold})")
            print("Restarting with Simulated Annealing...")
            
             # Run SA and get new population
            new_population = simulated_annealing_population(
                dist_matrix, 
                initial_temp, 
                cooling_rate, 
                stop_temp, 
                population_size
            )
            population = sorted(new_population, key=lambda x: x[1][0])[:population_size]
             # Mutation rate reset
            current_mutation_rate = initial_mutation_rate
            print("Population refreshed and mutation rate reset")

        new_population = []

        while len(new_population) < population_size:
            parent1 = tournament_selection(population, tournament_size)
            parent2 = tournament_selection(population, tournament_size)
            
            child = inver_over_crossover(parent1[0], parent2[0], crossover_iterations)
            
            if np.random.random() < current_mutation_rate:
                child = inversion_mutation(child)

            child_cost = tsp_cost(child, dist_matrix)
            new_population.append((child, child_cost))

        combined_population = np.array(new_population + population, dtype=object)
        sorted_indices = np.argsort([cost[0] for _, cost in combined_population])
        population = combined_population[sorted_indices][:population_size].tolist()

        new_best_solution, new_best_cost = min(population, key=lambda x: x[1][0])

         # Update the best result ever if needed
        if new_best_cost[0] < all_time_best_cost[0]:
            all_time_best_solution = new_best_solution
            all_time_best_cost = new_best_cost

        if new_best_cost[0] < best_cost[0]:
            best_solution, best_cost = new_best_solution, new_best_cost
            generations_without_improvement = 0
            current_mutation_rate = max(initial_mutation_rate, 
                                     current_mutation_rate * mutation_decrease_factor)
        else:
            generations_without_improvement += 1

        if generations_without_improvement > max_generations_without_improvement:
            current_mutation_rate *= mutation_increase_factor
            generations_without_improvement = 0

        if generation % 1000 == 0:
            print(f"Generation {generation}: "
                  f"Best current cost = {new_best_cost[0]:.2f} | "
                  f"Absolute best cost = {all_time_best_cost[0]:.2f} | "
                  f"Mutation rate = {current_mutation_rate:.3f} | "
                  f"Valid = {best_cost[1]}")
    
    return all_time_best_cost


## Vanuatu

In [ ]:

CITIES = pd.read_csv('./cities/vanuatu.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km

best_cost=evolutionary_algorithm_tsp_adaptive_mutation(
    DIST_MATRIX,
    num_generations=0, 
)
ic(best_cost,tsp_cost.calls)



0it [00:00, ?it/s]
ic| best_cost: (np.float64(1345.5449564733112), np.True_)
    tsp_cost.calls: 58196


((np.float64(1345.5449564733112), np.True_), 58196)

For vanuatu just applying simulated anneling to get the best solution is enough

## Italy

In [ ]:

CITIES = pd.read_csv('./cities/italy.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
best_cost=evolutionary_algorithm_tsp_adaptive_mutation(
    DIST_MATRIX,
    mutation_increase_factor=1.3,
    mutation_decrease_factor=0.7
)
ic(best_cost,tsp_cost.calls)


  0%|          | 10/10000 [00:00<03:37, 46.03it/s]

Generation 0: Best current cost = 4433.76 | Absolute best cost = 4433.76 | Mutation rate = 0.100 | Valid = True


 10%|█         | 1008/10000 [00:20<03:16, 45.71it/s]

Generation 1000: Best current cost = 4258.07 | Absolute best cost = 4258.07 | Mutation rate = 0.130 | Valid = True


 20%|██        | 2009/10000 [00:37<01:58, 67.36it/s]

...

Generation 8000: Best current cost = 4175.23 | Absolute best cost = 4175.23 | Mutation rate = 0.260 | Valid = True


 90%|█████████ | 9012/10000 [02:44<00:16, 60.62it/s]

Generation 9000: Best current cost = 4175.23 | Absolute best cost = 4175.23 | Mutation rate = 0.439 | Valid = True


100%|██████████| 10000/10000 [03:01<00:00, 55.24it/s]
ic| best_cost: (np.float64(4172.7626139164095), np.True_)
    tsp_cost.calls: 10872940


((np.float64(4172.7626139164095), np.True_), 10872940)

## Russia

In [ ]:

CITIES = pd.read_csv('./cities/russia.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
best_cost=evolutionary_algorithm_tsp_adaptive_mutation(DIST_MATRIX,mutation_threshold=1)
ic(best_cost,tsp_cost.calls)


  0%|          | 4/10000 [00:00<04:18, 38.63it/s]

Generation 0: Best current cost = 34150.88 | Absolute best cost = 34150.88 | Mutation rate = 0.100 | Valid = True


 10%|█         | 1006/10000 [00:23<03:20, 44.85it/s]

Generation 1000: Best current cost = 34029.74 | Absolute best cost = 34029.74 | Mutation rate = 0.100 | Valid = True


 20%|██        | 2004/10000 [00:45<03:10, 41.94it/s]

...

Population refreshed and mutation rate reset


 90%|█████████ | 9008/10000 [03:27<00:19, 51.52it/s]

Generation 9000: Best current cost = 34756.34 | Absolute best cost = 33954.59 | Mutation rate = 0.100 | Valid = True


100%|██████████| 10000/10000 [03:47<00:00, 43.90it/s]
ic| best_cost: (np.float64(33882.626394758365), np.True_)
    tsp_cost.calls: 2290980


((np.float64(33882.626394758365), np.True_), 2290980)

## Us

In [ ]:

CITIES = pd.read_csv('./cities/us.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
best_cost=evolutionary_algorithm_tsp_adaptive_mutation(
    DIST_MATRIX,
    num_generations=20000)
ic(best_cost,tsp_cost.calls)

  0%|          | 4/20000 [00:00<09:20, 35.66it/s]

Generation 0: Best current cost = 42437.24 | Absolute best cost = 42437.24 | Mutation rate = 0.100 | Valid = True


  5%|▌         | 1004/20000 [00:31<09:17, 34.05it/s]

Generation 1000: Best current cost = 40735.15 | Absolute best cost = 40735.15 | Mutation rate = 0.100 | Valid = True


 10%|█         | 2005/20000 [01:01<10:39, 28.16it/s]

...

Generation 18000: Best current cost = 41287.06 | Absolute best cost = 39614.05 | Mutation rate = 0.150 | Valid = True


 95%|█████████▌| 19005/20000 [10:12<00:31, 31.94it/s]

Generation 19000: Best current cost = 40149.29 | Absolute best cost = 39614.05 | Mutation rate = 0.338 | Valid = True


100%|██████████| 20000/20000 [10:47<00:00, 30.89it/s]
ic| best_cost: (np.float64(39614.05149170428), np.True_)
    tsp_cost.calls: 4465568


((np.float64(39614.05149170428), np.True_), 4465568)

## China

In [ ]:

CITIES = pd.read_csv('./cities/china.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
best_cost=evolutionary_algorithm_tsp_adaptive_mutation(
    DIST_MATRIX,
    num_generations=30000, 
)
ic(best_cost,tsp_cost.calls)


  0%|          | 4/30000 [00:00<33:56, 14.73it/s]

Generation 0: Best current cost = 63962.92 | Absolute best cost = 63962.92 | Mutation rate = 0.100 | Valid = True


  3%|▎         | 1004/30000 [01:11<25:26, 19.00it/s] 

Generation 1000: Best current cost = 61241.38 | Absolute best cost = 61241.38 | Mutation rate = 0.100 | Valid = True


  7%|▋         | 2004/30000 [02:05<23:16, 20.05it/s]

...

Generation 28000: Best current cost = 52463.86 | Absolute best cost = 52463.86 | Mutation rate = 0.100 | Valid = True


 97%|█████████▋| 29005/30000 [32:41<00:49, 20.25it/s]

Generation 29000: Best current cost = 52390.68 | Absolute best cost = 52390.68 | Mutation rate = 0.100 | Valid = True


100%|██████████| 30000/30000 [33:43<00:00, 14.83it/s]
ic| best_cost: (np.float64(52353.584217571246), np.True_)
    tsp_cost.calls: 13931136


((np.float64(52353.584217571246), np.True_), 13931136)

The proposed evolutionary algorithm addresses the traveling salesman problem (TSP) by integrating an adaptive mutation strategy and a restart mechanism based on Simulated Annealing (SA). Mutation is dynamically regulated: the rate increases if no improvement is observed for a given number of generations and decreases if progress is made, thus promoting a balance between exploration and exploitation.

When the mutation rate exceeds a predefined threshold, the algorithm initiates a restart phase via SA, regenerating the population to avoid convergence to local minima. The approach uses tournament selection, permutation-specific crossover techniques of the TSP (inver-over), and an inversion-based mutation. This combination allows the algorithm to progressively improve the quality of the solution.

Note: in order to obtain the same results, it may be necessary to restart the algorithm several times or to increase the gender number. The results shown on the screen are the highest values I obtained after several attempts